In [1]:
import string
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
pd.set_option('display.max_colwidth', 500)

# Global Functions and Variable

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_words = set(stopwords.words('indonesian'))
exceptions = {"sampai", "kurang", "lebih", "dari", "hari"}
custom_stop_words = stop_words - exceptions

In [ ]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [158]:
# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [159]:
def normalize_text(text, dictionary):
    text = re.sub(r'(\d+)\s*–\s*(\d+)', r'\1 sampai \2', text)
    text = text.replace('>', ' lebih dari ')
    text = text.replace('<', ' kurang dari ')
    
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [160]:
def preprocess_title_desc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

# Biofarma

In [161]:
biofarma_df = pd.read_csv('artikel/biofarma.csv')
biofarma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        108 non-null    object 
 1   tag          0 non-null      float64
 2   link         108 non-null    object 
 3   date         108 non-null    object 
 4   description  108 non-null    object 
 5   content      108 non-null    object 
dtypes: float64(1), object(5)
memory usage: 5.2+ KB


In [162]:
biofarma_df['source'] = 'Biofarma'
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 Cara Merawat Ginjal agar Sehat & Cegah Penyakit Ginjal,NaN,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,"Rabu, 3 September 2025",Kenali cara merawat ginjal agar terhindar dari gejala ginjal bermasalah. Pelajari ciri penyakit ginjal dan pencegahan penyakit ginjal apa saja sejak dini.,"Ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. Jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. Banyak penyakit ginjal apa saja yang berkembang tanpa gejala pada tahap awal, sehingga penting menjaga kesehatan ginjal sejak dini.\nBerikut ini adalah 5 cara merawat ginjal yang sederhana namun efektif untuk mencegah gejala ginjal bermasalah:\n1. Minum Air Putih yang Cukup\nAsupan cairan yang cukup mem...",Biofarma
1,7 Tips Untuk Menjaga Kesehatan Mata,NaN,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,"Jumat, 29 Agustus 2025","Tips menjaga kesehatan mata: nutrisi bergizi, kacamata pelindung, olahraga, hingga periksa rutin ke dokter","Menjaga kesehatan mata sangatlah penting karena mata berperan besar dalam hampir semua aktivitas sehari-hari, mulai dari belajar, bekerja, hingga berinteraksi dengan lingkungan sekitar. Jika kesehatan mata terabaikan, berbagai masalah bisa muncul, mulai dari iritasi ringan, infeksi, hingga penyakit serius yang berisiko menyebabkan kebutaan. Untuk itu, menjaga mata dapat dilakukan melalui pemeriksaan rutin ke dokter dan juga dengan membiasakan pola hidup sehat yang mendukung fungsi penglihata...",Biofarma
2,Jangan Sepelekan! 10 Penyebab Campak yang Sering Terabaikan,NaN,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,"Kamis, 28 Agustus 2025","Kenali faktor utama penyebab campak, gejala khasnya, dan cara pencegahan melalui imunisasi MMR","Penyebab Campak\nCampak adalah penyakit menular yang disebabkan oleh virus Morbili dan dapat menyebar dengan sangat cepat dari satu orang ke orang lain. Mekanisme utama penularannya adalah melalui percikan air liur (droplet) yang keluar dari mulut atau hidung penderita saat batuk, bersin, atau berbicara. Selain itu, seseorang juga dapat tertular jika menyentuh benda yang sudah terkontaminasi virus lalu menyentuh hidung, mulut, atau mata. Berikut adalah sepuluh faktor penyebab yang paling ber...",Biofarma
3,"Campak - Penyebab, Gejala, dan Pencegahannya",NaN,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,"Senin, 25 Agustus 2025",Campak bisa berbahaya bila dibiarkan. Periksa ke dokter sejak gejala awal muncul agar penanganan lebih tepat.,"Apa itu campak?\nCampak atau rubella adalah penyakit menular yang disebabkan oleh virus dari keluarga Paramyxoviridae dengan genus Morbillivirus. Penyakit ini menyerang saluran pernapasan terlebih dahulu, lalu menimbulkan gejala sistemik seperti demam, batuk, pilek, sakit tenggorokan, mata merah, hingga ruam menyeluruh di kulit. Campak termasuk penyakit yang sangat menular, bisa berpindah dari orang ke orang hanya melalui udara, percikan air liur, atau benda yang terkontaminasi virus.\nApa s...",Biofarma
4,Cara Mudah Menjaga Kesehatan Gigi dan Mulut agar Terhindar dari Penyakit,NaN,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,"Rabu, 20 Agustus 2025","Kesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk ke dalam tubuh, sekaligus organ yang berperan...","Apa itu Kesehatan Gigi dan Mulut?\nKesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk

In [163]:
biofarma_df.isnull().sum()

title            0
tag            108
link             0
date             0
description      0
content          0
source           0
dtype: int64

In [164]:
biofarma_df['tag'] = biofarma_df['tag'].fillna('Kesehatan Umum')

In [165]:
def preprocess_content_biofarma(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = re.sub(r'referensi[\s\S]*?(bagikan:|download|$)', '', text, flags=re.IGNORECASE) # Remove blok referensi ke akhir
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [166]:
biofarma_df['title'] = biofarma_df['title'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['description'] = biofarma_df['description'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['content'] = biofarma_df['content'].apply(lambda x: preprocess_content_biofarma(x))

In [167]:
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,"Rabu, 3 September 2025",kenal rawat ginjal hindar dari gejala ginjal masalah ajar ciri sakit ginjal cegah sakit ginjal,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...,Biofarma
1,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,"Jumat, 29 Agustus 2025",tips jaga sehat mata nutrisi gizi kacamata lindung olahraga periksa rutin dokter,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...,Biofarma
2,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,"Kamis, 28 Agustus 2025",kenal faktor utama sebab campak gejala khas cegah imunisasi mmr,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...,Biofarma
3,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,"Senin, 25 Agustus 2025",campak bahaya biar periksa dokter gejala muncul tangan lebih,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...,Biofarma
4,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,"Rabu, 20 Agustus 2025",sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...,Biofarma


# Klikdokter

In [168]:
klikdokter_df = pd.read_csv('artikel/klikdokter.csv')
klikdokter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        110 non-null    object
 1   tag          110 non-null    object
 2   link         110 non-null    object
 3   date         110 non-null    object
 4   description  103 non-null    object
 5   content      110 non-null    object
dtypes: object(6)
memory usage: 5.3+ KB


In [169]:
klikdokter_df['source'] = 'Klikdokter'
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,Jangan Panik! Kenali Rabies dan Pertolongan Pertama Saat Digigit Hewan Liar,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/pertolongan-pertama-saat-digigit-hewan-liar,30 Mei 2025,NaN,"Berita Kesehatan\nJangan Panik! Kenali Rabies dan Pertolongan Pertama Saat Digigit Hewan Liar\ndr. Dyah Novita, 30 Mei 2025\nDitinjau Oleh Tim Medis Klikdokter\nBagikan\nTahukah kamu bahwa rabies sangat berbahaya dan dapat mengancam nyawa? Penyakit ini menyerang sistem saraf pusat dan dapat menular melalui gigitan hewan yang terinfeksi, terutama anjing. Tapi jangan khawatir, dengan mengenal rabies, gejalanya, dan cara pencegahannya, Kamu bisa menjaga diri dan orang di sekitarmu tetap aman. ...",Klikdokter
1,Terapi Stem Cell untuk Pengobatan Penyakit Autoimun,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/terapi-stem-cell-untuk-pengobatan-penyakit-autoimun,29 Apr 2025,Penyakit autoimun seperti rheumatoid arthritis bisa menurunkan kualitas hidup. Terapi stem cell hadir sebagai harapan baru dalam pengobatannya.,"Berita Kesehatan\nTerapi Stem Cell untuk Pengobatan Penyakit Autoimun\ndr. Dyah Novita Anggraini, CBS, CIMI, 29 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nPenyakit autoimun seperti rheumatoid arthritis bisa menurunkan kualitas hidup. Terapi stem cell hadir sebagai harapan baru dalam pengobatannya.\n\nPenyakit autoimun, seperti rheumatoid arthritis menyebabkan peradangan pada sendi yang bisa menurunkan kualitas hidup. Umumnya, masalah kesehatan ini dikelola dengan ...",Klikdokter
2,"Terapi Stem Cell, Harapan Baru untuk Pengobatan Penyakit Degeneratif",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-pengobatan-penyakit-degeneratif-di-masa-depan,29 Apr 2025,Terapi stem cell kini menjadi harapan baru dalam menangani penyakit degeneratif akibat proses penuaan alami. Simak penjelasan lengkapnya di sini.,"Berita Kesehatan\nTerapi Stem Cell, Harapan Baru untuk Pengobatan Penyakit Degeneratif\ndr. Dyah Novita Anggraini, CBS, CIMI, 29 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nTerapi stem cell kini menjadi harapan baru dalam menangani penyakit degeneratif akibat proses penuaan alami. Simak penjelasan lengkapnya di sini.\nSemakin bertambah usia, penuaan akan terjadi secara alami. Tidak hanya kulit, organ penting di dalam tubuh seperti jantung, tulang, atau otak juga ak...",Klikdokter
3,"Stem Cell Tumbuhan dan Manusia Itu Berbeda, Ini Penjelasannya!",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-tumbuhan-dan-manusia-itu-berbeda-ini-penjelasannya,19 Apr 2025,"Baik tumbuhan maupun manusia memiliki stem cell., Nnamun keduanya memiliki perbedaan dalam struktur, fungsi, dan aplikasinya. Penasaran? Yuk, pelajari lewat ulasan berikut!","Berita Kesehatan\nStem Cell Tumbuhan dan Manusia Itu Berbeda, Ini Penjelasannya!\ndr. Dyah Novita Anggraini, CBS, CIMI, 19 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nBaik tumbuhan maupun manusia memiliki stem cell., Nnamun keduanya memiliki perbedaan dalam struktur, fungsi, dan aplikasinya. Penasaran? Yuk, pelajari lewat ulasan berikut!\nApa Itu Stem Cell?\nSecara umum stem cell adalah sel yang belum memiliki fungsi khusus, namun dapat membelah diri dan nantinya b...",Klikdokter
4,Keberagaman Model di Victoria’s Secret Fashion Show dan Mengenal Bentuk Tubuh Perempuan,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/mengenal-keberagaman-bentuk-tubuh-perempuan,17 Okt 2024,"Victoria's Secret ubah standar kecantikan! Show terbaru ini merayakan keberagaman bentuk tubuh, warna kulit, dan ukuran. Sebuah pesan kuat tentang self-love dan body positivity.","Berita Kesehatan\nKeberagaman Model di Victoria’s Secret Fashion Show dan Mengenal Bentuk Tubuh Perempuan\nChristovel Ramot, 17 Okt 2024\nDitinjau oleh dr. Dyah Novita Anggraini\nBagikan\nVictoria's Secret ubah s

In [170]:
klikdokter_df.isnull().sum()

title          0
tag            0
link           0
date           0
description    7
content        0
source         0
dtype: int64

In [171]:
klikdokter_df['description'] = klikdokter_df['description'].fillna('Tidak ada deskripsi')

In [172]:
def preprocess_content_klikdokter(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'^.*?Bagikan', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove pendahuluan artikel
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^artikel lainnya:.*', '', text, flags=re.MULTILINE) # Remove artikel lainnya
    text = re.sub(r'Yuk, download.*?Referensi', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove penutup artikel
    text = re.sub(r'Referensi.*', '', text, flags=re.IGNORECASE) # Remove referensi
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [173]:
klikdokter_df['title'] = klikdokter_df['title'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['description'] = klikdokter_df['description'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['content'] = klikdokter_df['content'].apply(lambda x: preprocess_content_klikdokter(x))

In [174]:
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,panik kenal rabies tolong gigit hewan liar,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/pertolongan-pertama-saat-digigit-hewan-liar,30 Mei 2025,deskripsi,tahu rabies bahaya ancam nyawa sakit serang sistem saraf pusat tular gigit hewan infeksi anjing khawatir kenal rabies gejala cegah jaga orang sekitar aman mari kenal rabies bahaya rabies rabies sakit infeksi serius serang sistem saraf pusat otak akibat lyssavirus sakit bahaya tular manusia gigit cakar hewan infeksi virus gigit hewan bawa virus rabies virus rabies masuk tubuh sebab infeksi potensi fatal mati hewan tular rabies manusia anjing kucing kera hewan liar rubah musang anjing liar sum...,Klikdokter
1,terapi stem cell obat sakit autoimun,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/terapi-stem-cell-untuk-pengobatan-penyakit-autoimun,29 Apr 2025,sakit autoimun rheumatoid arthritis turun kualitas hidup terapi stem cell hadir harap obat,sakit autoimun rheumatoid arthritis turun kualitas hidup terapi stem cell hadir harap obat sakit autoimun rheumatoid arthritis sebab adang sendi turun kualitas hidup sehat kelola konsumsi obat antiradang maju teknologi sehat stem cell yakin potensi salah obat sakit autoimun yuk simak ulas lengkap kenal stem cell sakit autoimun stem cell sel punca sel milik istimewa kembang jenis sel tubuh sel milik sifat utama selfrenewal belah banyak diferensiasi differentiation mampu stem cell kembang biak...,Klikdokter
2,terapi stem cell harap obat sakit degeneratif,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-pengobatan-penyakit-degeneratif-di-masa-depan,29 Apr 2025,terapi stem cell harap tangan sakit degeneratif akibat proses tua alami simak jelas lengkap,terapi stem cell harap tangan sakit degeneratif akibat proses tua alami simak jelas lengkap tambah usia tua alami kulit organ tubuh jantung tulang otak alami turun fungsi baik dari kondisi sebab sakit degeneratif teliti terapi stem cell potensi obat sakit degeneratif penasaran yuk simak ulas kenal terapi stem cell iring maju ilmu tahu teknologi dokter layan sel punca sel manfaat upaya layan sehat menteri sehat tetap atur menteri sehat nomor 32 2018 selenggara layan sel punca danatau sel tuju...,Klikdokter
3,stem cell tumbuh manusia beda jelas,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-tumbuhan-dan-manusia-itu-berbeda-ini-penjelasannya,19 Apr 2025,tumbuh manusia milik stem cell nnamun milik beda struktur fungsi aplikasi penasaran yuk ajar ulas,tumbuh manusia milik stem cell nnamun milik beda struktur fungsi aplikasi penasaran yuk ajar ulas stem cell stem cell sel milik fungsi khusus belah diferensiasi sel milik fungsi spesifik stem cell kenal sebut sel dasar sel punca sel induk manusia stem cell kembang otot saraf darah proses sembuh regenerasi jaring stem cell tumbuh sel meristem milik fungsi khusus diferensiasi stem cell manusia sel induk ujung akar batang jaring kambium sel tumbuh tumbuh baik artikel beda contoh terap exosome s...,Klikdokter
4,agam model victorias secret fashion show kenal bentuk tubuh perempuan,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/mengenal-keberagaman-bentuk-tubuh-perempuan,17 Okt 2024,victorias secret ubah standar cantik show baru raya agam bentuk tubuh warna kulit ukur pesan kuat selflove body positivity,victorias secret ubah standar cantik show baru raya agam bentuk tubuh warna kulit ukur pesan kuat selflove body positivity tanggal 15 oktober 2024 victorias secret gelar fashion show tahun dinantinanti tampil koleksi baru suasana beda banding tahuntahun usaha dulunya kenal standar ketat model tubuh langsing ubah arah kenal agam bentuk tubuh ajang gengsi show baru panggung victorias secret ramai model dari bentuk tubuh ukur warna kulit tanda langkah jalan dunia fashion inklusivitas victorias ...,Klikdokter


# Halodoc

In [175]:
halodoc_df = pd.read_csv('artikel/halodoc.csv')
halodoc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        72 non-null     object
 1   tag          72 non-null     object
 2   link         72 non-null     object
 3   date         69 non-null     object
 4   description  72 non-null     object
 5   content      69 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB


In [176]:
halodoc_df['source'] = 'halodoc'

In [177]:
halodoc_df.isnull().sum()

title          0
tag            0
link           0
date           3
description    0
content        3
source         0
dtype: int64

In [178]:
halodoc_df.dropna(inplace=True)

In [179]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,Kenalan dengan Manfaat Ceramide untuk Perawatan Kulit,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan-manfaat-ceramide-untuk-perawatan-kulit,15 September 2025,Ceramide mampu melindungi skin barrier secara efektif.,Advertisement\nTemukan lebih banyak\nHalodoc\nPerawatan Kulit\nmelembapkan\ntabir surya\nLotion\nLosion\nPerawatan\nCeradan\nlotion\nskincare\nMoisturizer\nSkincare\nhalodoc\nPerawatan kulit\nBeli vitamin dan suplemen\nHALODOC\nlosion\npelembap\nTabir surya\nmoisturizer\nMoisturizing\nPelembap\nperawatan kulit\nKenalan dengan Manfaat Ceramide untuk Perawatan Kulit\nKecantikan\nPerawatan Kulit\n7 menit\nDitinjau oleh \ndr. Erlian Dimas SpDVE\n 15 September 2025\nCeramide mampu melindungi ski...,halodoc
1,Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang: Penyebab dan Cara Mengatasinya,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-sebelah-kanan-bawah-sampai-ke-pinggang-penyebab-dan-cara-mengatasinya,15 September 2025,"Sakit perut sebelah kanan bawah sampai ke pinggang bisa jadi tanda masalah serius, kenali penyebabnya.","Advertisement\nSakit Perut Sebelah Kanan Bawah Sampai ke Pinggang: Penyebab dan Cara Mengatasinya\nSakit Pinggang\n5 menit\nDitinjau oleh \ndr. Fauzan Azhari SpPD\n 15 September 2025\nSakit perut sebelah kanan bawah sampai ke pinggang bisa jadi tanda masalah serius, kenali penyebabnya.\nDAFTAR ISI\nPenyebab Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang\nDiagnosis dan Pemeriksaan\nCara Mengatasi Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang\nKesimpulan\nFAQ\nSakit perut sebelah kan...",halodoc
2,"Mual dan Muntah, Kenali Gejala Alami Infeksi Astrovirus","Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-muntah-kenali-gejala-alami-infeksi-astrovirus,15 September 2025,"Infeksi astrovirus dapat menyebabkan gastroenteritis dengan berbagai gejala seperti mual, muntah, dan diare.","Advertisement\nMual dan Muntah, Kenali Gejala Alami Infeksi Astrovirus\nAstrovirus\nDemam\n4 menit\nDitinjau oleh \ndr. Budiyanto, MARS\n 15 September 2025\nInfeksi astrovirus dapat menyebabkan gastroenteritis dengan berbagai gejala seperti mual, muntah, dan diare.\nDAFTAR ISI\nGejala Infeksi Astrovirus\nMuntah: Gejala Umum Infeksi Astrovirus\nPenyebab Infeksi Astrovirus\nBagaimana Penularan Astrovirus Terjadi?\nDiagnosis Infeksi Astrovirus\nPengobatan Infeksi Astrovirus\nKapan Obat Antimun...",halodoc
3,Apa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,"Buah salak mengandung nutrisi vitamin A, B, C, zat besi, serta karbohidrat.","Advertisement\nApa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?\nMakanan Sehat\n7 menit\nDitinjau oleh \ndr. Fauzan Azhari SpPD\n 15 September 2025\nBuah salak mengandung nutrisi vitamin A, B, C, zat besi, serta karbohidrat.\nDaftar Isi:\nKandungan Vitamin dan Nutrisi pada Buah Salak\nManfaat Salak untuk Kesehatan Tubuh\nPotensi Efek Samping Konsumsi Salak\nTips Konsumsi Salak yang Aman dan Sehat\nKesimpulan\nFAQ\nSalak adalah buah tropis yang populer di Indonesia dan dikenal deng...",halodoc
4,Ketahui Fungsi Rongga Mulut dalam Sistem Pencernaan Manusia,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi-rongga-mulut-dalam-sistem-pencernaan-manusia,15 September 2025,"Rongga mulut berfungsi untuk mengunyah, memproses makanan secara kimiawi, dan menyalurkan makanan ke dalam lambung.","Advertisement\nKetahui Fungsi Rongga Mulut dalam Sistem Pencernaan Manusia\nMulut dan Gigi\n7 menit\nDitinjau oleh \ndr. Budiyanto, MARS\n 15 September 2025\nRongga mulut berfungsi untuk mengunyah, memproses makanan secara kimiawi, dan menyalurkan makanan ke dalam lambung.\nDAFTAR ISI\nFungsi Utama Rongga Mulut\nAnatomi dan Fungsi Mulut pada Manusia\nCara Menjaga Fungsi Rongga Mulut\nMasalah Kesehatan yang Umum Terjadi pada Mulut\nRongga mulut adalah bagian awal dari s

In [183]:
def preprocess_content_halodoc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^.*?daftar isi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'mau tahu.*?referensi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'referensi.*$', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [186]:
halodoc_df['title'] = halodoc_df['title'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['description'] = halodoc_df['description'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['content'] = halodoc_df['content'].apply(lambda x: preprocess_content_halodoc(x))

In [187]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,kenal manfaat ceramide awat kulit,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan-manfaat-ceramide-untuk-perawatan-kulit,15 September 2025,ceramide lindung skin barrier efektif,manfaat ceramide kulit rekomendasi skincare kandung ceramide produk ceramide hasil optimal efek samping perhati tips pilih produk ceramide dokter ceramide jenis asam lemak nama lipid kandung sel kulit manusia bentuk 50 persen lapis kulit luar epidermis asam lemak kerap jadi salah bahan produk awat kulit kerja lindung halang papar radikal bebas potensi rusak kulit ceramide efektif tahan lapis lindung kulit skin barrier kandung ceramide tingkat hidrasi kulit lebih kenyal lembab radiance by web...,halodoc
1,sakit perut belah kanan sampai pinggang sebab atas,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-sebelah-kanan-bawah-sampai-ke-pinggang-penyebab-dan-cara-mengatasinya,15 September 2025,sakit perut belah kanan sampai pinggang tanda serius kenal sebab,sebab sakit perut belah kanan sampai pinggang diagnosis periksa atas sakit perut belah kanan sampai pinggang simpul faq sakit perut belah kanan sampai pinggang kondisi nyeri muncul perut kanan jalar pinggang sakit sifat ringan berat gantung sebab anatomi perut kanan isi organ usus buntu usus salur kemih wanita ovarium kanan tuba falopi pinggang kanan kait otot tulang ginjal sakit perut belah kanan sampai pinggang asal dari organ beda sebab sakit perut belah kanan sampai pinggang sebab timbul...,halodoc
2,mual muntah kenal gejala alami infeksi astrovirus,"Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-muntah-kenali-gejala-alami-infeksi-astrovirus,15 September 2025,infeksi astrovirus sebab gastroenteritis gejala mual muntah diare,gejala infeksi astrovirus muntah gejala infeksi astrovirus sebab infeksi astrovirus tular astrovirus diagnosis infeksi astrovirus obat infeksi astrovirus obat antimuntah komplikasi infeksi astrovirus cegah infeksi astrovirus dokter astrovirus jenis virus rna sebab infeksi salur cerna anakanak infeksi seringkali timbul gejala diare mual muntah demam ringan sakit perut bahaya infeksi astrovirus sebab dehidrasi bayi anak gejala infeksi astrovirus infeksi astrovirus gejala mencret darah mual mun...,halodoc
3,kandung vitamin buah salak,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,buah salak kandung nutrisi vitamin a b c zat besi karbohidrat,kandung vitamin nutrisi buah salak manfaat salak sehat tubuh potensi efek samping konsumsi salak tips konsumsi salak aman sehat simpul faq salak buah tropis populer indonesia kenal kulit sisik rupa kulit ular buah milik manis asam sepat unik lezat salak kaya nutrisi manfaat sehat rutin konsumsi buah sehat kandung vitamin dalam penasaran saa kandung manfaat buah salak sehat tubuh tahu lengkap kandung vitamin nutrisi buah salak salak buah milik kulit rupa sisik manis daging buah asal dari indo...,halodoc
4,tahu fungsi rongga mulut sistem cerna manusia,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi-rongga-mulut-dalam-sistem-pencernaan-manusia,15 September 2025,rongga mulut fungsi kunyah proses makan kimiawi salur makan lambung,fungsi utama rongga mulut anatomi fungsi mulut manusia jaga fungsi rongga mulut sehat mulut rongga mulut dari sistem cerna manusia kali anggap sederhana padaha fungsil mulut milik peran kompleks vital proses cerna makan dari kunyah hasil enzim mulut gerbang utama tentu tubuh proses nutrisi komunikasi pasti lebih sulit rongga mulut tahu fungsi mulut fungsi utama rongga mulut mulut milik fungsi utama tubuh terima makan mulut makan masuk tubuh kecap lidah milik reseptor nikmat makan kunyah maka...,halodoc


# Alodokter

In [ ]:
# miripin sama yang sebelum"nya

# Detik Health

In [ ]:
# miripin sama yang sebelum"nya

# Kompas Health

In [ ]:
# miripin sama yang sebelum"nya